In [3]:
from langchain.tools import tool
import os
from typing import Annotated, Sequence, TypedDict
from langchain.tools import tool
from langchain_community.vectorstores import FAISS
from langchain_text_splitters  import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langchain_community.document_loaders import PyPDFLoader
from langgraph.checkpoint.memory import InMemorySaver  
from pydantic import BaseModel, Field
from typing import List, Literal
from typing_extensions import TypedDict
from langchain.agents.structured_output import ProviderStrategy
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent,AgentState

c:\Users\aiear_vpvq6b8\Desktop\pro\hack\hask\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["AZURE_OPENAI_API_KEY"]=os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION")
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]=os.getenv("AZURE_OPENAI_DEPLOYMENT")
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    "azure_openai:gpt-4.1",
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
)

In [3]:
USER_DATABASE = {
    "2871513": {
        "name": "Arun",
        "employee_id": "2871513",
        "ultimatix_login_last": "12 hours ago",
        "domain_account_locked": {"state": False},
        "ultimatix_account_locked": {
            "state": True,
            "reason": "Wrong password attempts more than 2 times"
        },
        "domain_region": "India",
        "asset_id": None,
        "email": "arun.s@tcs.com",
        "bgv_status": "Pending",
        "project": "Bench",
        "grade": "Y"
    },
    "2987452": {
        "name": "Priya",
        "employee_id": "2987452",
        "ultimatix_login_last": "2 days ago",
        "domain_account_locked": {
            "state": True,
            "reason": "Password expired"
        },
        "ultimatix_account_locked": {"state": False},
        "domain_region": "India",
        "asset_id": "AS12345",
        "email": "priya.k@tcs.com",
        "bgv_status": "Completed",
        "project": "Retail Transformation",
        "grade": "A1"
    },
    "2764019": {
        "name": "Rahul",
        "employee_id": "2764019",
        "ultimatix_login_last": "Just now",
        "domain_account_locked": {"state": False},
        "ultimatix_account_locked": {"state": False},
        "domain_region": "Europe",
        "asset_id": "DE99881",
        "email": "rahul.m@tcs.com",
        "bgv_status": "Completed",
        "project": "Banking L2 Support",
        "grade": "A2"
    },
    "2890077": {
        "name": "Sneha",
        "employee_id": "2890077",
        "ultimatix_login_last": "5 hours ago",
        "domain_account_locked": {
            "state": True,
            "reason": "Multiple wrong login attempts"
        },
        "ultimatix_account_locked": {
            "state": True,
            "reason": "Authenticator not registered"
        },
        "domain_region": "USA",
        "asset_id": None,
        "email": "sneha.p@tcs.com",
        "bgv_status": "Initiated",
        "project": "Onboarding",
        "grade": "Y"
    }
}


In [5]:
class CustomState(AgentState):
    employee_id: str

In [6]:
@tool
def get_employee_details(runtime: ToolRuntime[None, CustomState]) -> str:
    """Get employee's account details, account status, and onboarding progress. Use this to provide personalized help based on the employee's actual situation."""
    employee_id = runtime.state.get("employee_id")
    
    if not employee_id or employee_id not in USER_DATABASE:
        return "Employee not found in system."
    
    employee = USER_DATABASE[employee_id]
    
    # Build detailed employee context
    account_status = []
    
    if employee['ultimatix_account_locked']['state']:
        account_status.append(f"🔒 Ultimatix Account LOCKED: {employee['ultimatix_account_locked']['reason']}")
    else:
        account_status.append("✅ Ultimatix Account: Active")
    
    if employee['domain_account_locked']['state']:
        account_status.append(f"🔒 Domain Account LOCKED: {employee['domain_account_locked'].get('reason', 'Unknown')}")
    else:
        account_status.append("✅ Domain Account: Active")
    
    details = f"""
EMPLOYEE: {employee['name']} (ID: {employee_id})
Email: {employee['email']}
Region: {employee['domain_region']}
Project: {employee['project']}
Grade: {employee['grade']}
BGV Status: {employee['bgv_status']}
Last Ultimatix Login: {employee['ultimatix_login_last']}
Asset ID: {employee['asset_id'] or 'Not assigned'}

ACCOUNT STATUS:
{chr(10).join(account_status)}
"""
    
    return details

get_employee_details()

TypeError: 'StructuredTool' object is not callable

In [6]:
tools=[get_employee_details]

In [7]:
class replytype(TypedDict):
    answer:bool =Field(
        ...,
        description="Lookup tool that verifies if a question is answerable from the employee JSON fields"
    )
    reason: str = Field(...,
                        description="Please provide the reason behind your decision, whether it is a yes or a no.")

In [8]:
agent = create_agent(
    llm,
    tools,
    system_prompt = """You are a Knowledge-Base Lookup Agent.

INPUT YOU WILL RECEIVE:
- A natural-language question asked by the user.
- The employee’s complete knowledge-base record (a JSON object containing all known fields for that employee).

YOUR SINGLE PURPOSE:
Identify whether the user’s question can be answered using ONLY the fields present in the provided employee JSON.

You DO NOT answer the user’s question.  
You ONLY validate whether the knowledge to answer it exists.

----------------------------------------------------
EVALUATION RULES
----------------------------------------------------

1. FIELD AVAILABILITY CHECK
   - Determine if the question is asking for information that corresponds to any field (or nested field) in the employee JSON.
   - Use semantic understanding. Do NOT rely on exact text matching.
     Examples:
       “Is my domain locked?” → maps to domain_account_locked.state
       “What project am I working on?” → maps to project
       “Where am I based?” → maps to domain_region
       “Is my Ultimatix locked?” → maps to ultimatix_account_locked.state

2. IF INFORMATION EXISTS IN THE JSON:
      Respond EXACTLY:
        {
          "result": "YES"
        }

3. IF INFORMATION DOES NOT EXIST:
      Respond EXACTLY:
        {
          "result": "NO",
          "reason": "Requested information is not available in the employee knowledge base."
        }

4. FORBIDDEN ACTIONS:
   - Do NOT provide the actual answer to the question.
   - Do NOT rewrite or interpret the employee’s data.
   - Do NOT include additional fields or commentary.
   - Do NOT include explanations beyond the allowed "reason".

5. OUTPUT FORMAT:
   - Always return strictly valid JSON.
   - Only include:
        - "result": "YES" or "NO"
        - "reason" (only when result = "NO")
   - No extra fields, no arrays, no text outside the JSON.

6. STRICTNESS:
   - “YES” only if the exact information needed to answer the question fully and directly exists somewhere inside the employee JSON.
   - If partially related but insufficient → return “NO”.

----------------------------------------------------
Your job ends after determining YES or NO. 
Do not attempt any other task.
----------------------------------------------------
""",
    checkpointer=InMemorySaver(),state_schema=CustomState,
    response_format=ProviderStrategy(replytype))

In [10]:
config = {"configurable": {"thread_id": "2"}}
result = agent.invoke(
    {
        "messages": "what is domain region",
        "employee_id": "2871513"
    },
    config
)
result

{'messages': [HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}, id='fc66a6ba-f5b2-4c51-83a8-ce066b2f40d0'),
  AIMessage(content='{"answer":false,"reason":"Requested information is not available in the employee knowledge base."}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 557, 'total_tokens': 582, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-CiccwmFMrlh1niIahnPgeTuDoGGlQ', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'saf

{'messages': [HumanMessage(content='what is my domain region', additional_kwargs={}, response_metadata={}, id='cd2bcf46-daf2-4d97-81a3-6a651acc6a80'),
  AIMessage(content='{"answer":true,"reason":""}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 558, 'total_tokens': 573, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-CibC8DK0IHxXHigUgh5zlJkC4NJFc', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'viole

In [39]:
EMBED = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
top_k=10

In [40]:
def make_retriever_tool_from_text(file, name, desc):
    docs = PyPDFLoader(file).load()
    chunks = RecursiveCharacterTextSplitter(
        chunk_size=500, 
        chunk_overlap=50
    ).split_documents(docs)
    
    vs = FAISS.from_documents(chunks, EMBED)
    retriever = vs.as_retriever(search_type="similarity",  # or "mmr" for maximal marginal relevance
        search_kwargs={"k": top_k})

    @tool  # Use @tool decorator without arguments
    def tool_func(query: str) -> str:
        """Retrieve documents based on query."""
        print(f"📚 Using tool: {name}")
        results = retriever.invoke(query)
        return "\n\n".join(doc.page_content for doc in results)
    
    # Set name and description after decoration
    tool_func.name = name
    tool_func.description = desc
    
    return tool_func

In [43]:
internal_tool_1=make_retriever_tool_from_text(
    file="TCS_Intern_Onboarding_Instructions.pdf",
    name="tcs_onboarding_agent",
    desc="""This tool is an expert assistant for TCS intern onboarding. It answers queries strictly using information from the TCS Intern Onboarding Instructions PDF. The document covers essential onboarding workflows for new joiners, including Ultimatix activation, UxApps and Authenticator setup, profile completion (PAN, Aadhaar, bank details, address, photo), permanent access requests, TCS email activation timelines, daily timesheet procedures, mandatory iEvolve compliance courses, and support contacts.Use this tool when queries relate to TCS onboarding, setup activities, compliance tasks, account activation, or navigation of official processes."""
)
internal_tool_1

StructuredTool(name='tcs_onboarding_agent', description='This tool is an expert assistant for TCS intern onboarding. It answers queries strictly using information from the TCS Intern Onboarding Instructions PDF. The document covers essential onboarding workflows for new joiners, including Ultimatix activation, UxApps and Authenticator setup, profile completion (PAN, Aadhaar, bank details, address, photo), permanent access requests, TCS email activation timelines, daily timesheet procedures, mandatory iEvolve compliance courses, and support contacts.Use this tool when queries relate to TCS onboarding, setup activities, compliance tasks, account activation, or navigation of official processes.', args_schema=<class 'langchain_core.utils.pydantic.tool_func'>, func=<function make_retriever_tool_from_text.<locals>.tool_func at 0x00000243AD7A09A0>)

In [44]:
tools=[internal_tool_1]

In [78]:
class ContactInfo(TypedDict):
    answer: str = Field(
        ...,
        description="Provide a short, warm, and empathetic reply tailored to the user's current emotion and situation. Keep the tone friendly, simple, and reassuring for freshers."
    )
    call_llm_judge: bool = Field(
        ...,
        description="Indicate whether this issue requires escalation after 3 unresolved troubleshooting attempts (true = escalate)."
    )
    emotion: str = Field(
        ...,
        description="Identify the dominant emotion expressed by the user dynamically (e.g., confused, stressed, nervous, excited, frustrated, embarrassed, etc.)."
    )


In [79]:
from langchain.agents import create_agent
sop_agent = create_agent(
    llm,
    tools,
    system_prompt="""You are the TCS Onboarding Assistant helping new employees and interns (mostly freshers).

SCOPE

* Help with TCS onboarding topics:

  * Ultimatix activation & login issues
  * UxApps / Authenticator setup
  * TCS email activation and access
  * Profile updates (PAN, Aadhaar, address, bank, photo)
  * Access requests, timesheets, iEvolve / compliance courses
  * Support contacts and escalation paths


FRESHER MINDSET – CRITICAL

* Assume the user is a complete fresher:

  * They may not know what “domain account”, “BGV”, “timesheet”, “iEvolve”, etc. mean.
  * Avoid internal jargon, or briefly explain it in simple words the first time you use it.
* First, understand what they are asking:

  * If their question is unclear, ask ONE simple clarifying question before giving steps.
  * Rephrase their issue in simple words to confirm: e.g., “So you’re not able to log in to Ultimatix after entering your password, right?”
* Always start with empathy and reassurance:

  * Example: “Hi Arun, I know onboarding can feel confusing at first, but don’t worry, we’ll sort this out together.”
* Make the user feel safe and not judged:

  * Normalize mistakes like password errors: “This happens to many new joiners, it’s okay.”

STYLE

* Be warm, friendly, and concise.
* Use short answers: 3–4 sentences or a few bullet points.
* Prefer simple, direct language and short sentences.
* Avoid long paragraphs; break things into small bullets or numbered steps.
* Try to keep your reply shorter than the user’s last message when possible.
* Sound like a helpful guide, not a formal email.

STEP-BY-STEP TROUBLESHOOTING – DO NOT DUMP EVERYTHING
For each issue:

1. Start with empathy + quick summary of what you understood.
2. Give only 1–2 immediate checks or actions (very small steps).
3. After those steps, ask exactly ONE clear follow-up question, such as:

   * “Did this step work for you?”
   * “Can you confirm what you see on the screen now?”
   * “Would you like to continue to the next step?”
4. Wait for the user’s response before giving more steps.
5. Do NOT dump the entire full procedure at once, even if the solution is long.
6. For long flows, explicitly ask for permission to continue:

   * “We have a few more small steps. Can we continue to the next step now?”

FOLLOW-UP QUESTION RULES
* Ask follow-up **only** if:
  * You genuinely need info to proceed, OR
  * More steps depend on the user’s result.
* Do NOT ask follow-up questions:
  * Just to fill space
  * When you don’t have knowledge to continue
  * When escalation is required instead

EMOTION DETECTION & EMPATHY — DYNAMIC RESPONSE
Detect the user’s emotional tone from their message (e.g., confused, nervous, excited, embarrassed, angry, stressed, curious, overwhelmed, happy).
Always begin with a tailored empathetic response that matches their specific emotion — not a fixed set of examples.
If the emotion is unclear, default to a soft, supportive tone.
Normalize their feeling, especially as they are freshers who may feel lost:
Examples (to be used only if appropriate based on detected emotion):
Nervous: “I know starting something new can feel overwhelming, but I’ll guide you step by step.”
Embarrassed: “Please don’t worry — many new joiners face this too, and it’s perfectly okay to ask.”
Confused: “Thanks for letting me know. I’ll explain this in a simple way so it’s easier.”
Excited: “Love your enthusiasm! Let’s make sure everything goes smoothly.”
After acknowledging their emotion:
Move quickly to small, practical actions so they feel progress and control.
Empathy must feel human and conversational, not scripted or repetitive.

TROUBLESHOOTING ATTEMPTS & ESCALATION
* Every time the user says the step did not work = 1 failed attempt.
* After each failed attempt:
  1. Appreciate their effort — “Thanks for trying that.”
  2. Change the next step based on what they said — no repeating.
* If the user remains unhappy or the issue remains unresolved **after THREE attempts**:
  * Stop troubleshooting.
  * Escalate by replying with:
    `call_llm_judge: true`
  * Also include a short empathetic line such as:
    “I’m sorry it’s still not working even after multiple tries. I’ll help escalate this to someone who can fix it directly.”


OUT-OF-SCOPE

* If the user asks about non-onboarding topics (math, news, CEO, coding, etc.), say:

  * “I’m set up only to help with TCS onboarding topics, so I can’t answer that.”
* If information is not in your knowledge base:

  * “I don’t have information about that in my current knowledge base.”
* Never mention PDFs, retrieval tools, or implementation details.

MEMORY & CONTEXT
* Reference progress from earlier — show continuity.
* Track:
  * Number of troubleshooting attempts (for escalation logic)
  * What has already been tried, so you don’t repeat the same steps.
   """
,
    checkpointer=InMemorySaver(),
    response_format=ProviderStrategy(ContactInfo)
)

In [86]:
config_1 = {"configurable": {"thread_id": "3"}}
result = sop_agent.invoke(
    {
        "messages": "still i have issue"
    },
    config_1
)

In [87]:
result

{'messages': [HumanMessage(content='i have issue in ultimatix olgin', additional_kwargs={}, response_metadata={}, id='f3bee3c7-6a44-4b05-9563-3bb83ba9ad78'),
  AIMessage(content='{"answer":"Hi! It seems you\'re having trouble logging into Ultimatix. Let’s resolve this step by step. Can you tell me if you’re facing issues with entering your username/password or is it something else?","emotion":"confused","call_llm_judge":false}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 1359, 'total_tokens': 1423, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-CicRN5ggySKNco5kWEyasAmthELmB', 'prompt_filter_results': [{'prompt_index': 0, 'conten

In [88]:
config_1

{'configurable': {'thread_id': '3'}}